# OpenAI Log Probabilities Explorer

**Authors:** [Yanan Cai](https://github.com/YananCai) and [Scott Hanselman](https://github.com/shanselman) with minor edits to add comments for learning and handle pip install for set up in a dev container.

This notebook demonstrates how to explore log probabilities from OpenAI models deployed through Azure AI Foundry.

In [ ]:
%%capture

#Install required packages and surpress all outputs.
%pip install -q --upgrade pip
%pip install -q openai termcolor

In [ ]:
# Import necessary libraries

import os
from openai import AzureOpenAI
import math

In [ ]:
# Make sure environment variables are set

assert os.getenv("AOAI_KEY") is not None

In [22]:
# Initialize Azure OpenAI client with credentials from environment variables
client = AzureOpenAI(
    api_key=os.getenv("AOAI_KEY"),
    api_version="2024-10-21",
    azure_endpoint=os.getenv("AOAI_ENDPOINT")
)

# Specify the deployment name for the GPT-4o model
deployment_name = os.getenv("AOAI_DEPLOYMENT")


def get_model_response(client, prompt, deployment_name):
    """
    Get a response from the Azure OpenAI model with token log probabilities.

    Args:
        client: AzureOpenAI client instance
        prompt: User prompt to send to the model
        deployment_name: Name of the deployed model

    Returns:
        tuple: (response_str, logprobs) where:
            - response_str: The model's text response
            - logprobs: List of tuples (selected_token, top_logprobs) where top_logprobs 
                       contains the top 5 token candidates with their probabilities
    """
    messages = [{"role": "user", "content": prompt}]
    try:
        # Request completion with log probabilities enabled
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=50,  # Limit response length
            logprobs=True,  # Enable log probability output
            top_logprobs=5,  # Return top 5 token alternatives at each position
            # Control randomness (0=deterministic, 1=creative)
            temperature=0.7,
        )

        # Extract the generated text response
        response_str = response.choices[0].message.content

        # Parse log probabilities for each token
        logprobs = []
        tokens = response.choices[0].logprobs.content

        for token_info in tokens:
            token = token_info.token
            top_logprobs = []

            # Convert log probabilities to actual probabilities (0-1 range)
            for logprob in token_info.top_logprobs:
                prob = math.exp(logprob.logprob)  # exp(log_prob) = prob
                top_logprobs.append((logprob.token, prob))

            logprobs.append((token, top_logprobs))

    except Exception as e:
        print(f"An exception occurred: {e}")

    return response_str, logprobs


def show_and_write_probs(logprobs):
    """
    Display token probabilities in a formatted, color-coded table layout.

    Shows the selected token and its top 5 alternatives with probabilities as percentages.
    Displays up to 5 token columns per row for readability.

    Args:
        logprobs: List of tuples (selected_token, top_logprobs) from get_model_response()
    """
    from termcolor import colored

    # Define color scheme for output
    probs_color = "light_magenta"  # Alternative tokens
    token_color = "light_green"  # Token index headers
    selected_token_color = "cyan"  # Actually selected token (marked with =)

    num_tokens = len(logprobs)
    num_top_probs = len(logprobs[0][1])

    # Build data structure for display
    data = []
    token_index = 0

    for selected_token, top_logprobs in logprobs:
        token_data = []

        # Header: token index and the selected token
        display_token = repr(selected_token)[1:-1]  # Remove quotes from repr
        token_data.append(f"  {token_index}: {display_token}")

        # Add each alternative token with its probability
        for token, prob in top_logprobs:
            display_token = repr(token)[1:-1]

            # Mark the actually selected token with '=' instead of ':'
            if token == selected_token:
                token_data.append(f"{display_token:>12}= {prob * 100:7.2f}")
            else:
                token_data.append(f"{display_token:>12}: {prob * 100:7.2f}")

        data.append(token_data)
        token_index += 1

    # Display tokens in groups of 5 columns for readability
    tables_per_line = 5

    for i in range(0, num_tokens, tables_per_line):
        # Print each row (header + top probs) across multiple token columns
        for j in range(num_top_probs + 1):
            for k in range(i, min(i + tables_per_line, num_tokens)):
                if j < len(data[k]):
                    if j == 0:
                        # First row: token index header
                        print(
                            colored(f"{data[k][j]:<28}", token_color), end="")
                    else:
                        # Probability rows: highlight selected token
                        if data[k][j][12] == "=":
                            print(
                                colored(f"{data[k][j]:<28}", selected_token_color), end="")
                        else:
                            print(
                                colored(f"{data[k][j]:<28}", probs_color), end="")
                else:
                    # Empty cell padding
                    print(" " * 25, end="")
            print()
        print()  # Blank line between row groups

In [23]:
prompt = "Who is Harry Potter?"
response, token_probs = get_model_response(client, prompt, deployment_name)
print(response)
show_and_write_probs(token_probs)

Harry Potter is a fictional character and the protagonist of the *Harry Potter* book series, written by British author J.K. Rowling. The series follows Harry's journey as a young wizard who discovers his magical heritage and attends Hogwarts School of Witchcraft and
  0: Harry                    1:  Potter                  2:  is                      3:  a                       4:  fictional             
       Harry=   99.88             Potter=   99.03                 is=  100.00                  a=   53.12          fictional=   99.90       
          **:    0.10              James:    0.97                  ,:    0.00                the:   46.88            beloved:    0.05       
           *:    0.01                 **:    0.00             refers:    0.00                 an:    0.00             famous:    0.04       
           ":    0.00                Pot:    0.00                   :    0.00                one:    0.00            popular:    0.00       
          \n:    0.00       